In [2]:
from gdb2pgsql import transfer_gdb_to_postgis
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine,text 
import leafmap

In [4]:
# Define the paths and layer name (comment out either one gdb_path when not in use)
#gdb_path = "data/iB1000.gdb"
gdb_path = r"C:\Users\Steve_Lau\Desktop\LS Training\ls_project1\iB1000.gdb"

# Define the db connection parameters
username = "postgres"
password = "12345"
host = "localhost"
dbname = "iB1000"
port = "5432"

pg_connection = f"PG:host={host} port={port} dbname={dbname} user={username} password={password}"
engine = create_engine(f"postgresql://{username}:{password}@{host}:{port}/{dbname}")

In [5]:
transfer_gdb_to_postgis(gdb_path, pg_connection)

Layer: UtilityNumber, Type: None
Layer: BUILDINGRELATEBUILDINGNAME, Type: None
Layer: SITENAME, Type: None
Layer: SITERELATESITENAME, Type: None
Layer: Street_Code, Type: None
Layer: BUILDINGNAME, Type: None
Layer: ADDRESSRELATEBUILDINGSITE, Type: None
Layer: ADDRESS, Type: None
Layer: BSLineAnno, Type: Multi Polygon
Layer: BSPointAnno, Type: Multi Polygon
Layer: BSPolygonAnno, Type: Multi Polygon
Layer: Building, Type: 3D Multi Polygon
Layer: BuildingAccessoryLine, Type: 3D Multi Line String
Layer: BuildingAnno, Type: Multi Polygon
Layer: BuildingAnnoHouseNo, Type: Multi Polygon
Layer: BuiltStructureLine, Type: 3D Multi Line String
Layer: BuiltStructurePoint, Type: 3D Point
Layer: BuiltStructurePolygon, Type: 3D Multi Polygon
Layer: CartoBuildingLine, Type: 3D Multi Line String
Layer: ProposedInfraAnno, Type: Multi Polygon
Layer: ProposedInfraStructureLine, Type: 3D Multi Line String
Layer: ProposedInfraStructurePoint, Type: 3D Point
Layer: Site, Type: 3D Multi Polygon
Layer: SiteAnno

In [17]:
table_name_1 = "BUILDINGNAME"
table_name_2 = "BUILDINGRELATEBUILDINGNAME"
table_name_3 = "building"
sql_1 = text(f"SELECT * FROM {table_name_1}")
sql_2 = text(f"SELECT * FROM {table_name_2}")
sql_3 = text(f"SELECT * FROM {table_name_3}")

In [23]:
# Read the tables into DataFrames
df_1 = pd.read_sql(sql_1, con=engine.connect())
df_2 = pd.read_sql(sql_2, con=engine.connect())

# Read the tables with geometry into DataFrames
gdf_1 = gpd.read_postgis(sql_3, con=engine.connect(), geom_col="shape") 

df_1.head()

,objectid,lastupdatedate,buildingnameid,englishbuildingname,chinesebuildingname,namestatus,st_code
0,1,2023-09-02 16:00:00+00:00,1101368238,Sham Tseng Village Tsz Tong,深井村祠堂,E,68225.0
1,2,2023-09-02 16:00:00+00:00,1101366428,Tsing Fai Tong Aqua Privy,深井清快塘旱廁,E,68213.0
2,3,2023-09-02 16:00:00+00:00,1101366358,Sham Tseng Sun Village Aqua Privy,深井新村旱廁,E,64079.0
3,4,2023-09-02 16:00:00+00:00,1101366588,Sham Tseng Village Supply Pump House,深井鄉村食水泵房,E,68225.0
4,5,2023-09-02 16:00:00+00:00,1101366678,None,排棉角村龍子苑,E,64075.0


In [24]:
# Select only the "BUILDINGNAMEID", "ENGLISHBUILDINGNAME", and "CHINESEBUILDINGNAME" columns from df_1
df_1_subset = df_1[["buildingnameid",
                  "englishbuildingname",
                  "chinesebuildingname"]]

# Merge df2 with the subset of df1 on the "BUILDINGNAMEID" column
df_3 = pd.merge(df_2, df_1_subset, on="buildingnameid")

# Display the first few rows of the merged DataFrame
df_3.head()

,objectid,lastupdatedate,buildingnameid,buildingid,englishbuildingname,chinesebuildingname
0,2,2023-09-02 16:00:00+00:00,1101367973,1100000458,Chuen Lung Village Aqua Privy,川龍村旱廁
1,3,2023-09-02 16:00:00+00:00,1101367804,1100004056,None,木棉下村公所何氏家祠
2,5,2023-09-02 16:00:00+00:00,1101368243,1100011094,Mount Haven Block 5,曉峰園第５座
3,6,2023-09-02 16:00:00+00:00,1101367784,1100003706,Yi Pei Chun Toilet,二陂圳廁所
4,7,2023-09-02 16:00:00+00:00,1101365720,1100002338,Lo Wai Village Office,老圍村公所


In [25]:
# Select only the "BUILDINGID", "ENGLISHBUILDINGNAME", and "CHINESEBUILDINGNAME" columns from df3
df_3_subset = df_3[["buildingid",
                  "englishbuildingname",
                  "chinesebuildingname"]]

# Merge gdf1 with the subset of df3 on the "BUILDINGID" column
gdf_2 = pd.merge(gdf_1,
                df_3_subset,
                on="buildingid")

# Display the first few rows of the merged DataFrame
gdf_2.head()

,objectid,lastupdatedate,buildingid,typeofbuildingblock,baselevel,rooflevel,baseleveldatasource,roofleveldatasource,buildingstatus,certainty,shape_length,shape_area,shape,englishbuildingname,chinesebuildingname
0,6,2024-05-07 16:00:00+00:00,1910247693,T,5.3,120.0,10.0,10.0,E,1,134.739147,1175.061936,"MULTIPOLYGON (((834979.859 812118.914, 834945....",Viva Place,偉華匯
1,8,2024-05-04 16:00:00+00:00,1910247673,P,NaN,NaN,NaN,NaN,P,1,576.375163,4219.597921,"MULTIPOLYGON (((838683.841 832108.401, 838684....",Podium,平台
2,9,2024-05-04 16:00:00+00:00,1910247670,P,NaN,NaN,NaN,NaN,P,1,211.508289,1867.085492,"MULTIPOLYGON (((838802.481 832136.004, 838802....",Podium,平台
3,10,2024-05-04 16:00:00+00:00,1910247669,P,NaN,NaN,NaN,NaN,P,1,267.948414,2513.780813,"MULTIPOLYGON (((838774.334 832051.217, 838798....",Podium,平台
4,11,2024-05-04 16:00:00+00:00,1910247632,T,NaN,NaN,NaN,NaN,P,1,187.242046,1123.065147,"MULTIPOLYGON (((838755.868 832088.787, 838752....",RESIDENTIAL RECREATIONAL FACILITIES,住客康樂設施


In [ ]:
#shp_path = "data/building.shp"

#Convert the datetimes to strings
#gdf2['LASTUPDATEDATE'] = gdf2['LASTUPDATEDATE'].dt.strftime('%Y-%m-%d')

#gdf2.to_file(shp_path, encoding='utf-8')

In [26]:
# Keep only relevant columns
gdf_3 = gdf_2.loc[:, ("buildingid",
                    "typeofbuildingblock",
                    "baselevel",
                    "rooflevel",
                    "buildingstatus",
                    "englishbuildingname",
                    "chinesebuildingname")]

gdf_3.head(30)

,buildingid,typeofbuildingblock,baselevel,rooflevel,buildingstatus,englishbuildingname,chinesebuildingname
0,1910247693,T,5.3,120.0,E,Viva Place,偉華匯
1,1910247673,P,NaN,NaN,P,Podium,平台
2,1910247670,P,NaN,NaN,P,Podium,平台
3,1910247669,P,NaN,NaN,P,Podium,平台
4,1910247632,T,NaN,NaN,P,RESIDENTIAL RECREATIONAL FACILITIES,住客康樂設施
5,1910247623,T,NaN,NaN,P,Scenic Tower 1,優景閣第１座
6,1910247622,T,NaN,NaN,P,SCENIC TOWER 6,優景閣第６座
7,1910247621,T,NaN,NaN,P,MARINA TOWER 1,漾景閣第１座
8,1910247620,T,NaN,NaN,P,MARINA TOWER 2,漾景閣第２座
9,1910247619,T,NaN,NaN,P,MARINA TOWER 3,漾景閣第３座
